# Лабораторная 3 — ELMo + BiLSTM NER



In [6]:
from pathlib import Path
import sys

LAB_DIR = Path.cwd()
if not (LAB_DIR / 'train_elmo_lstm.py').exists():
    candidate = LAB_DIR / 'NLP_LAB_3'
    if (candidate / 'train_elmo_lstm.py').exists():
        LAB_DIR = candidate.resolve()
        sys.path.append(str(LAB_DIR))
        
DATA_DIR = (LAB_DIR / '..' / 'data').resolve()
MODELS_DIR = (LAB_DIR / '..' / 'models').resolve()
REPORTS_DIR = LAB_DIR / 'reports'

print(f'Рабочая директория: {LAB_DIR}')
print(f'Папка с данными: {DATA_DIR}')
print(f'Папка с моделями: {MODELS_DIR}')
print(f'Отчёты будут сохранены в: {REPORTS_DIR}')

Рабочая директория: /Users/il_dimas/Documents/Programming_projects/NLP_LABS/NLP_LAB_3
Папка с данными: /Users/il_dimas/Documents/Programming_projects/NLP_LABS/data
Папка с моделями: /Users/il_dimas/Documents/Programming_projects/NLP_LABS/models
Отчёты будут сохранены в: /Users/il_dimas/Documents/Programming_projects/NLP_LABS/NLP_LAB_3/reports


## Установка зависимостей


In [7]:
!python3 -m pip install --user -r requirements.txt

## Загрузка датасета и ELMo-модели

In [8]:
import subprocess
import urllib.request
import zipfile
import shutil

DATA_REPO = DATA_DIR / 'Detailed-NER-Dataset-RU'
if not DATA_REPO.exists():
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    subprocess.run(['git', 'clone', 'https://github.com/AlexKly/Detailed-NER-Dataset-RU.git', str(DATA_REPO)], check=True)

MODEL_DIR = MODELS_DIR / 'ruwikiruscorpora_tokens_elmo_1024_2019'
ZIP_PATH = MODELS_DIR / 'ruwikiruscorpora_tokens_elmo_1024_2019.zip'
MODEL_URL = 'https://vectors.nlpl.eu/repository/20/195.zip'

if not MODEL_DIR.exists():
    MODELS_DIR.mkdir(parents=True, exist_ok=True)
    urllib.request.urlretrieve(MODEL_URL, ZIP_PATH)
    with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
        zf.extractall(MODELS_DIR / 'ruwikiruscorpora_tokens_elmo_1024_2019')
    ZIP_PATH.unlink(missing_ok=True)

print('DATASET AND MODEL IS LOADED !!!')

DATASET AND MODEL IS LOADED !!!


## Обучение модели

In [10]:
import subprocess
import sys

command = [
    sys.executable,
    'train_elmo_lstm.py',
    '--epochs', '5',
    '--batch-size', '12',
    '--hidden-size', '256'
]
print(' '.join(command))
subprocess.run(command, check=True, cwd=LAB_DIR)

/Library/Developer/CommandLineTools/usr/bin/python3 train_elmo_lstm.py --epochs 5 --batch-size 12 --hidden-size 256


/Library/Developer/CommandLineTools/usr/bin/python3: can't open file '/Users/il_dimas/Documents/Programming_projects/NLP_LABS/NLP_LAB_3/train_elmo_lstm.py': [Errno 2] No such file or directory


CalledProcessError: Command '['/Library/Developer/CommandLineTools/usr/bin/python3', 'train_elmo_lstm.py', '--epochs', '5', '--batch-size', '12', '--hidden-size', '256']' returned non-zero exit status 2.

## Анализ метрик

In [11]:
import json
from pprint import pprint

metrics_path = REPORTS_DIR / 'metrics.json'
if metrics_path.exists():
    with metrics_path.open('r', encoding='utf-8') as fp:
        metrics = json.load(fp)
    print('Лучшие валидационные метрики:')
    pprint(metrics.get('best_val'))
    print('Тестовая выборка:')
    pprint(metrics.get('test'))
else:
    print('Не найден metrics.json. Убедитесь, что обучение завершилось успешно.')

Лучшие валидационные метрики:
{'f1': 0.848180677540778,
 'loss': 0.08443144975072304,
 'precision': 0.8600508905852418,
 'recall': 0.8366336633663366}
Тестовая выборка:
{'f1': 0.8397291196388261,
 'loss': 0.07965631827236934,
 'precision': 0.8571428571428571,
 'recall': 0.8230088495575221}


## Внешние тексты

In [12]:
external_path = REPORTS_DIR / 'external_predictions.json'
if external_path.exists():
    with external_path.open('r', encoding='utf-8') as fp:
        samples = json.load(fp)
    for sample in samples:
        print('=' * 80)
        print(sample['text'])
        print('Предсказанные сущности:', sample['entities'] or '—')

Мэр Москвы Сергей Собянин объявил о запуске новой линии МЦД между Одинцово и Лобней.
Предсказанные сущности: [['Москвы', 'CITY'], ['Сергей', 'FIRST_NAME'], ['Собянин', 'LAST_NAME'], ['Одинцово', 'DISTRICT'], ['Лобней', 'DISTRICT']]
РБК со ссылкой на источники сообщил, что Минфин готовит пакет налоговых льгот для IT-компаний.
Предсказанные сущности: —
Газпром договорился с властями Санкт-Петербурга о строительстве второй ветки «Лахта центру».
Предсказанные сущности: [['Санкт-Петербурга', 'CITY']]
Губернатор Краснодарского края Вениамин Кондратьев открыл обновленный аэропорт Сочи.
Предсказанные сущности: [['Краснодарского', 'REGION'], ['Вениамин', 'FIRST_NAME'], ['Кондратьев', 'LAST_NAME'], ['Сочи', 'CITY']]
Госдума рассмотрит проект о введении электронных гарантий для экспортеров сельхозпродукции.
Предсказанные сущности: —
Компания Яндекс представила новый сервис доставки продуктов в Казани и Нижнем Новгороде.
Предсказанные сущности: [['Казани', 'CITY'], ['Нижнем Новгороде', 'CITY']]
Су